In [53]:
import json
import uuid
import shapely
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
from shapely.geometry import Point, LineString
from geojson import Point, Feature, FeatureCollection, dump
from data_models import CreateBaseRunsLiftsGraph, GetConnections, CreateRunsLiftsGraph, CreateRunsLiftsNodesGraph, CreateNodesGraph

dir = "/Users/sebastian/Documents/PrivateProjects/SkiNavData/Data/"
dir_resorts = "/Users/sebastian/Documents/PrivateProjects/SkiNavData/SkiResorts/"


# get from https://epsg.io/
# UoM must be meters!
# crs = "27561" France
crs = 21781

In [54]:
features = ["id", "name", "ski_area_id", "connection_type", "difficulty",
            "duration", "distance", "geometry", "start", "end", "status"]

## Load and select ski areas

In [3]:
filename = "ski_areas.geojson"
file = open(dir + filename)
ski_areas = gpd.read_file(file)
ski_areas = ski_areas[['id', 'name', 'runConvention']]
ski_areas = ski_areas.dropna(subset='name')

In [55]:
name = 'Grindelwald'
print(ski_areas[ski_areas.name.str.contains(name)])
ski_area_id = ski_areas[ski_areas.name.str.contains(name)].id.values.tolist()
ski_area_name = ski_areas[ski_areas.name.str.contains(name)].name.values.tolist()
print(ski_area_id, ski_area_name)

                                           id  \
8    17b19c745f7d69545421bdb985f0b981abfc70ae   
407  1871bd97633b45248e93e03d11f44d94caae83fc   
414  1d111e0379650f6a9186eaa3d21dadf973340c88   

                                                  name runConvention  
8            Grindelwald - Männlichen (Schlittelpiste)        europe  
407                                  Grindelwald First        europe  
414  Grindelwald - Wengen (Kleine Scheidegg - Männl...        europe  
['17b19c745f7d69545421bdb985f0b981abfc70ae', '1871bd97633b45248e93e03d11f44d94caae83fc', '1d111e0379650f6a9186eaa3d21dadf973340c88'] ['Grindelwald - Männlichen (Schlittelpiste)', 'Grindelwald First', 'Grindelwald - Wengen (Kleine Scheidegg - Männlichen)']


In [56]:
ski_area_id = ski_area_id[2]
ski_area_name = ski_area_name[2]
print(ski_area_id, ski_area_name)

1d111e0379650f6a9186eaa3d21dadf973340c88 Grindelwald - Wengen (Kleine Scheidegg - Männlichen)


## Load ski runs

In [57]:
def get_id(data):
    json_file = json.loads(data)
    if len(json_file) > 0:
        id = json_file[0].get("properties").get("id")
    else:
        id = pd.NA
    return id

In [58]:
filename = "runs.geojson"
file = open(dir + filename)
runs = gpd.read_file(file)

In [60]:
runs["ski_area_id"] = runs.skiAreas.apply(lambda x: get_id(x))
runs = runs[runs.ski_area_id==ski_area_id]
runs = runs[runs.elevationProfile.notnull()]
len(runs)

121

### Change coordinates to CRS France

unit of measurement: metric

In [61]:
# set new CRS
runs = runs.to_crs(crs)
runs["uses"] = runs.uses.apply(lambda x: x[0])
# get length of run
runs["distance"] = runs.geometry.length
# get start coordinate
runs["start"] = runs.geometry.apply(lambda x: Point(x.coords[0]))
# get end coordinate
runs["end"] = runs.geometry.apply(lambda x: Point(x.coords[-1]))
# set connection type to run
runs["connection_type"] = "run"
# calculate duration be assuming an average speed of 25km/h or 7m/s
runs["duration"] = runs["distance"] / 7
# filter for downhill ski runs
runs = runs[runs.uses=="downhill"]
# return defined features
runs = runs[features]

## Load ski lifts

In [64]:
filename = "lifts.geojson"
file = open(dir + filename)
lifts = gpd.read_file(file)

In [65]:
lifts["ski_area_id"] = lifts.skiAreas.apply(lambda x: get_id(x))
lifts = lifts[lifts.ski_area_id==ski_area_id]
len(lifts)

20

### Change coordinates to CRS France

unit of measurement: metric

In [66]:
# set new CRS
lifts = lifts.to_crs(crs)
# get length of lift
lifts["distance"] = lifts.geometry.length
# get start coordinate
lifts["start"] = lifts.geometry.apply(lambda x: Point(x.coords[0]))
# get end coordinate
lifts["end"] = lifts.geometry.apply(lambda x: Point(x.coords[-1]))
# generate lift linesstring
lifts["geometry"] = lifts.geometry.apply(lambda x: LineString([x.coords[0], x.coords[-1]]))
# set connection type to lift
lifts["connection_type"] = "lift"
lifts = lifts.rename(columns={"liftType": "lift_type"})
# set difficulty to lift
lifts["difficulty"] = "lift"
# calcualte average speed in m/s (distance / duration)
mean_speed = lifts[lifts["duration"].notnull()]["distance"].sum() / lifts[lifts["duration"].notnull()].duration.sum()
lifts["duration"] = np.where(lifts["duration"].isna(), (lifts["distance"] / mean_speed), lifts["duration"])
# return defined features
lifts = lifts[features]

## Concatenate run and lift data

In [ ]:
data = pd.concat([lifts, runs])

# quality test if data is correct
data.geometry.plot(figsize=(15, 15))
plt.show()

## Generate points dataset

In [68]:
# transform linestrings to points
points = data.geometry.apply(lambda x: x.coords).rename("point").explode().reset_index().rename(columns={"index": "Index"})
# reset index of original data, needed for merging
data = data.reset_index().rename(columns={"index": "Index"})
# generate unique id for each point
unique_id = []
for i in range(len(points)):
    unique_id.append(str(uuid.uuid4()))
# set point_id
points["point_id"] = unique_id
# merge points data with original data on Index
points = points.merge(data[["id", "name", "Index", "difficulty", "duration",
                            "distance", "connection_type", "start", "end", "status"]], how="left", on="Index")
# transform points data to GeoDataFrame
points = gpd.GeoDataFrame(points, geometry=[Point(x) for x in points.point]).rename(columns={"id": "run_id"})
# get start points of all runs
start_points = points.groupby("run_id").first().reset_index()
start_points["point_type"] = "start"
# get end points of all runs
end_points = points.groupby("run_id").last().reset_index()
end_points["point_type"] = "end"
# combine start and end points
start_end_points = pd.concat([start_points, end_points], axis=0).sort_values(by="run_id")
points["points_by_run"] = points.groupby("run_id").point_id.transform("count")

## Create RunsLiftsGraph and NodesGraph

In [69]:
# create runsliftsgraph
RunsLiftsGraph = CreateBaseRunsLiftsGraph(points)

# get connected points and assing new unique node it
RunsLiftsGraph_tmp = GetConnections(RunsLiftsGraph, start_end_points)

# create RunsLiftsGraph
RunsLiftsGraph = CreateRunsLiftsGraph(RunsLiftsGraph_tmp)

# iterate through RunLifts, calculate segment length and keep nodes, start and end points
RunsLiftsNodesGraph = CreateRunsLiftsNodesGraph(RunsLiftsGraph_tmp)

# create NodesGraph
NodesGraph_tmp = CreateNodesGraph(RunsLiftsNodesGraph)
NodesGraph = {}
NodesGraph["features"] = NodesGraph_tmp

/Users/sebastian/VirtualEnvironments/SkiNavApp/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/sebastian/VirtualEnvironments/SkiNavApp/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


## Generate ski resort manifest

## Add new ski resort to manifest

In [70]:
# set versions and construct filename
RunsLiftsGraphVersion = "1.0"
NodesGraphVersion = "1.0"

In [71]:
with open(dir_resorts + "SkiResortManifest.json", "r", encoding="utf8") as json_file:
    SkiResortManifest = json.load(json_file)

SkiResortManifest["resorts"][ski_area_id] = {"ResortName": ski_area_name,
                                                "RunsLiftsGraphVersion": RunsLiftsGraphVersion,
                                                "NodesGraphVersion": NodesGraphVersion,
                                                "RunsLiftsGraphUpdated": datetime.date.today().strftime("%d/%m/%Y"),
                                                "NodesGraphUpdated": datetime.date.today().strftime("%d/%m/%Y")}

SkiResortManifest

{'resorts': {'68b126bc3175516c9263aed7635d14e37ff360dc': {'ResortName': 'Les Trois Vallées',
   'RunsLiftsGraphVersion': '1.0',
   'NodesGraphVersion': '1.0',
   'RunsLiftsGraphUpdated': '26/04/2025',
   'NodesGraphUpdated': '26/04/2025'},
  '1d111e0379650f6a9186eaa3d21dadf973340c88': {'ResortName': 'Grindelwald - Wengen (Kleine Scheidegg - Männlichen)',
   'RunsLiftsGraphVersion': '1.0',
   'NodesGraphVersion': '1.0',
   'RunsLiftsGraphUpdated': '26/04/2025',
   'NodesGraphUpdated': '26/04/2025'}}}

In [72]:
with open (dir_resorts + "SkiResortManifest.json", "w", encoding="utf8") as json_file:
    json_file.write(json.dumps(SkiResortManifest, ensure_ascii=False, indent=2))

## Save RunsLiftsGraph and NodesGraph files

In [73]:
# add new folder for ski resort if it not already exists
folder_path = Path(f"{dir_resorts}{ski_area_id}")
folder_path.mkdir(parents=True, exist_ok=True)

# add files
print(f"{dir_resorts}{ski_area_id}/RunsLiftsGraph_v{RunsLiftsGraphVersion}.geojson")
with open (f"{dir_resorts}{ski_area_id}/RunsLiftsGraph_v{RunsLiftsGraphVersion}.geojson", "w", encoding="utf8") as geojson_file:
    geojson_file.write(json.dumps(RunsLiftsGraph, ensure_ascii=False, indent=2))

print(f"{dir_resorts}{ski_area_id}/NodesGraph_v{NodesGraphVersion}.json")
with open (f"{dir_resorts}{ski_area_id}/NodesGraph_v{NodesGraphVersion}.json", "w", encoding="utf8") as json_file:
    json_file.write(json.dumps(NodesGraph, ensure_ascii=False, indent=2))

/Users/sebastian/Documents/PrivateProjects/SkiNavData/SkiResorts/1d111e0379650f6a9186eaa3d21dadf973340c88/RunsLiftsGraph_v1.0.geojson
/Users/sebastian/Documents/PrivateProjects/SkiNavData/SkiResorts/1d111e0379650f6a9186eaa3d21dadf973340c88/NodesGraph_v1.0.json


In [74]:
# swap lat and lon for Mapbox

with open(f"{dir_resorts}{ski_area_id}/RunsLiftsGraph_v{RunsLiftsGraphVersion}.geojson", "r", encoding="utf8") as geojson:
    geojson_swapped = json.load(geojson)


for feature in geojson_swapped['features']:
    if feature['geometry']['type'] == 'LineString':
        new_coords = []
        for coord in feature['geometry']['coordinates']:
            lat, lon, *rest = coord
            new_coord = [lon, lat] + rest
            new_coords.append(new_coord)
        feature['geometry']['coordinates'] = new_coords

geojson_swapped

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'run_id': '4fa6f19e164cc25315dac8e95fd8e13e2263aeb8',
    'run_name': 'Wengen LWM',
    'connection_type': 'lift',
    'difficulty': 'lift',
    'status': 'operating',
    'duration': 300.0,
    'point_id': ['ae20b9f9-9418-45f6-9628-542d11c25a57',
     '758a601e-7c36-4d73-bf22-318a5b46853d']},
   'geometry': {'type': 'LineString',
    'coordinates': [[2.8437057761250517, 49.155111508709076, 1279.0],
     [2.8641435946974174, 49.16201294187768, 2217.0]]}},
  {'type': 'Feature',
   'properties': {'run_id': '82461e98ce71ec14d2c845c7614311681e625947',
    'run_name': 'Männlichenbahn',
    'connection_type': 'lift',
    'difficulty': 'lift',
    'status': 'operating',
    'duration': 522.0,
    'point_id': ['58a6b8c6-b25b-4785-9ea0-892622ff933e',
     '22b98de9-c0b3-47a8-a9bc-80795182a4c3']},
   'geometry': {'type': 'LineString',
    'coordinates': [[2.945676077280669, 49.17349772701236, 941.0],
     [2.904376

In [75]:
print(f"{dir_resorts}{ski_area_id}/RunsLiftsGraph_swapped_v{RunsLiftsGraphVersion}.geojson")
with open (f"{dir_resorts}{ski_area_id}/RunsLiftsGraph_swapped_v{RunsLiftsGraphVersion}.geojson", "w", encoding="utf8") as geojson_file:
    geojson_file.write(json.dumps(geojson_swapped, ensure_ascii=False, indent=2))

/Users/sebastian/Documents/PrivateProjects/SkiNavData/SkiResorts/1d111e0379650f6a9186eaa3d21dadf973340c88/RunsLiftsGraph_swapped_v1.0.geojson
